In [61]:
%reload_ext autoreload
%autoreload 2
import sys
from dotenv import load_dotenv
import os
load_dotenv()

ROOT = os.getenv("ROOT")
sys.path.append(ROOT)

from src.nlp.regressor import BertRegressorPipeline
from src.data_loader import load_reviews
from src.processing import reviews_processing
from src.nlp.classifier import BERTClassifier

In [62]:
df = load_reviews(category="All_beauty", frac=0.001)
df = reviews_processing(df=df, clean_text=False)
sub = df.rename(columns={"rating": "target", "review_input": "text"})[["target", "text"]]

In [63]:
bert_pipeline = BertRegressorPipeline(df=sub)

In [64]:
bert_pipeline._prepare_data()

[16:04:03] | INFO  | split : {'train_set': 421, 'eval_set': 141, 'test_set': 140}                  ]8;id=50631;file:///Users/lucanyckees/Desktop/my-repos/recommendation_system/src/nlp/regressor.py\regressor.py]8;;\:]8;id=75954;file:///Users/lucanyckees/Desktop/my-repos/recommendation_system/src/nlp/regressor.py#116\116]8;;\

In [65]:
bert_pipeline._setup_model()

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertRegressor were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['cls_layer1.bias', 'cls_layer1.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.

In [66]:
bert_pipeline.train()

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]Python(95864) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Token indices sequence length is longer than the specified maximum sequence length for this model (646 > 512). Running this sequence through the model will result in indexing errors
/Users/lucanyckees/Desktop/my-repos/recommendation_system/venv/lib/python3.11/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch:   0%|          | 0/2 [03:45<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
bert_pipeline.evaluate()

Python(95056) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/lucanyckees/Desktop/my-repos/recommendation_system/venv/lib/python3.11/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors

nan

In [ ]:
import torch
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
predicted_label = []
actual_label = []
with torch.no_grad():
    for input_ids, attention_mask, target in bert_pipeline.validation_loader:
        input_ids, attention_mask, target = (
            input_ids.to(device),
            attention_mask.to(device),
            target.to(device),
        )
        output = bert_pipeline.model(input_ids, attention_mask)

        predicted_label += output
        actual_label += target
        print(output, target)
        break

Python(95203) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], device='mps:0') tensor([1., 4., 5., 5., 5., 5., 5., 5., 5., 5., 2., 2., 3., 4., 5., 5.],
       device='mps:0')


In [ ]:
actual_label

[tensor(1., device='mps:0'),
 tensor(4., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(2., device='mps:0'),
 tensor(2., device='mps:0'),
 tensor(3., device='mps:0'),
 tensor(4., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(2., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(1., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(1., device='mps:0'),
 tensor(3., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(4., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(5., device='mps:0'),
 tensor(1., de